In [1]:
import sys, os, glob, re, random, json, math, subprocess
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, ConfusionMatrixDisplay, f1_score, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset

c:\speech-text-recognition\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)

In [9]:
RAW_DIR = "../data/raw"         # put RAVDESS .wav here
INTERIM_DIR = "data/interim" # transcripts, merged CSVs

In [14]:
import glob, os
files = glob.glob(RAW_DIR + "/**/*.wav", recursive=True)
print("Found", len(files), "wav files. Example:", files[:3])


Found 3000 wav files. Example: ['../data/raw\\Actor_01\\03-01-01-01-01-01-01.wav', '../data/raw\\Actor_01\\03-01-01-01-01-02-01.wav', '../data/raw\\Actor_01\\03-01-01-01-02-01-01.wav']


In [19]:
import pandas as pd

# mapping numbers → emotion names
EMO_MAP = {
    '01':'neutral','02':'calm','03':'happy','04':'sad',
    '05':'angry','06':'fearful','07':'disgust','08':'surprised'
}

def parse_ravdess_filename(path: str):
    """
    Decode one RAVDESS filename into its meaning.
    Example filename: 03-01-05-02-02-01-12.wav
    """
    base = os.path.basename(path)
    parts = base.replace(".wav", "").split("-")
    if len(parts) != 7:
        return None
    return {
        "path": path,
        "emotion_id": parts[2],
        "emotion": EMO_MAP.get(parts[2], "unknown"),
        "intensity": parts[3],    # 01=normal, 02=strong
        "statement": parts[4],    # 01 / 02 (two sentences)
        "repetition": parts[5],   # 01 / 02
        "actor": parts[6]
    }

# apply function to every file
rows = [parse_ravdess_filename(p) for p in files]
rows = [r for r in rows if r is not None]  # drop any failed parses

df_meta = pd.DataFrame(rows)
print(df_meta.shape)
df_meta.head()


(3000, 7)


,path,emotion_id,emotion,intensity,statement,repetition,actor
0,../data/raw\Actor_01\03-01-01-01-01-01-01.wav,01,neutral,01,01,01,01
1,../data/raw\Actor_01\03-01-01-01-01-02-01.wav,01,neutral,01,01,02,01
2,../data/raw\Actor_01\03-01-01-01-02-01-01.wav,01,neutral,01,02,01,01
3,../data/raw\Actor_01\03-01-01-01-02-02-01.wav,01,neutral,01,02,02,01
4,../data/raw\Actor_01\03-01-02-01-01-01-01.wav,02,calm,01,01,01,01


In [25]:
df_meta.to_csv("../data/interim/metadata.csv", index=False)
print(" Saved to data/interim/metadata.csv")


 Saved to data/interim/metadata.csv


In [29]:
import subprocess

def ok(cmd):
    try:
        r = subprocess.run(cmd, capture_output=True, text=True)
        return r.returncode == 0
    except FileNotFoundError:
        return False

print("ffmpeg?", ok(["ffmpeg", "-version"]))
print("whisper?", ok(["whisper", "--help"]))


ffmpeg? False
whisper? False
